# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA+'*/*/*.parquet'))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag_1=x['Adj Close'].shift(1),
        hi_lo_range=x['High'] - x['Low']
    )
).assign(
    returns=lambda x: (x['Close'] / x['Close_lag_1']) - 1
))

/var/folders/6g/zpbgfkbn6l54k9rbsjjvvzlh0000gn/T/ipykernel_3920/2710884593.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
pd_feat = dd_feat.compute()
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns
ticker,,,,,,,,,,,,,,
OXY,2023-01-03,62.299999,62.950001,60.110001,61.049999,59.891586,13104000,Energy,Oil & Gas Exploration & Production,2023,NaN,NaN,2.840000,NaN
OXY,2023-01-04,59.939999,61.840000,59.700001,61.110001,59.950451,12238000,Energy,Oil & Gas Exploration & Production,2023,61.049999,59.891586,2.139999,0.000983
OXY,2023-01-05,61.119999,62.730000,60.750000,62.220001,61.039387,10692800,Energy,Oil & Gas Exploration & Production,2023,61.110001,59.950451,1.980000,0.018164
OXY,2023-01-06,63.410000,64.629997,62.939999,63.750000,62.540352,9832700,Energy,Oil & Gas Exploration & Production,2023,62.220001,61.039387,1.689999,0.024590
OXY,2023-01-09,64.870003,66.580002,63.980000,64.080002,62.864094,12519800,Energy,Oil & Gas Exploration & Production,2023,63.750000,62.540352,2.600002,0.005176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2018-12-24,185.000000,188.380005,182.610001,182.750000,176.080261,1007900,Information Technology,Application Software,2018,185.809998,179.028564,5.770004,-0.016468
INTU,2018-12-26,184.130005,192.860001,183.070007,192.800003,185.763474,1987700,Information Technology,Application Software,2018,182.750000,176.080261,9.789993,0.054993
INTU,2018-12-27,189.559998,197.389999,187.839996,197.240005,190.041428,2730900,Information Technology,Application Software,2018,192.800003,185.763474,9.550003,0.023029


In [12]:
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(10).mean()
pd_feat.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns,rolling_avg_return
ticker,,,,,,,,,,,,,,,
INTU,2018-12-24,185.000000,188.380005,182.610001,182.750000,176.080261,1007900,Information Technology,Application Software,2018,185.809998,179.028564,5.770004,-0.016468,-0.012172
INTU,2018-12-26,184.130005,192.860001,183.070007,192.800003,185.763474,1987700,Information Technology,Application Software,2018,182.750000,176.080261,9.789993,0.054993,-0.006344
INTU,2018-12-27,189.559998,197.389999,187.839996,197.240005,190.041428,2730900,Information Technology,Application Software,2018,192.800003,185.763474,9.550003,0.023029,-0.005530
INTU,2018-12-28,197.740005,199.490005,194.449997,195.850006,188.702164,1340200,Information Technology,Application Software,2018,197.240005,190.041428,5.040009,-0.007047,-0.005881
INTU,2018-12-31,197.369995,198.759995,195.429993,196.850006,189.665649,1074200,Information Technology,Application Software,2018,195.850006,188.702164,3.330002,0.005106,-0.001768


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?<br>
**Answer:** 
No, it was not necessary, as both approaches would work well in this case.

+ Would it have been better to do it in Dask? Why?<br>
**Answer:** Given the size of the data, it may be more efficient to use Dask due to its parallel processing capabilities. This approach allows you to work with data that exceeds your system's memory limitations while fully leveraging your hardware resources.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.